# DASC 5300, Fall2024, University of Texas at Arlington
# PA4 "Implementing the Go Fish Card Game in Python"


## **Academic Honesty**
This assignment must be done individually and independently. You must implement the whole assignment by yourself. Academic dishonesty is not tolerated.

## **Requirements**

1.   When you work on this assignment, you should make a copy of this notebook in Google Colab. This can be done using the option `File > Save a copy in Drive` in Google Colab.

2.  To submit your assignment, download your Colab into a .ipynb file. This
can be done using the option `Download > Download .ipynb` in Google Colab. Submit the downloaded .ipynb file/ .zip into the PA4 entry in Canvas.


## **Description**

Implementing the Go Fish card game using stacks and queues in Python. The program should simulate a game between two players, where each player draws cards from a central deck to collect sets of matching cards. The game ends when the deck is empty, and the player with the most sets at the end of the game wins. The program should use QUEUE to represent each player's hand and a STACK to represent the central deck of cards. The game should be playable in the command-line interface, with the option to specify the number of players and the number of cards dealt to each player at the start of the game."

In [20]:
import random
from collections import deque
from dataclasses import dataclass
from enum import Enum
from typing import Deque, Optional, Union

In [21]:
class CardRank(Enum):
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    SIX = "6"
    SEVEN = "7"
    EIGHT = "8"
    NINE = "9"
    TEN = "10"
    JACK = "J"
    QUEEN = "Q"
    KING = "K"
    ACE = "A"

    # def __str__(self):
    #     return self.value


class CardSuit(Enum):
    HEARTS = "♥"
    DIAMONDS = "♦"
    CLUBS = "♣"
    SPADES = "♠"

In [22]:
# Card class to combine rank and suit
@dataclass
class Card:
    def __init__(self, rank: CardRank, suit: CardSuit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank.value}{self.suit.value}"

    def __repr__(self):
        return f"Card(rank={self.rank}, suit={self.suit})"

    def __eq__(self, card):
        return self.rank == card.rank and self.suit == card.suit


In [23]:
class Player:
    def __init__(self, hand: Optional[Deque[Card]] = None):
        self._hand = hand if hand else deque()
        self._sets = set()

    # Check and collect sets of 4 matching ranks
    def count_sets(self):
        rank_counts = {}
        sets = 0

        # Count the occurrences of each rank
        for card in self._hand:
            rank_counts[card.rank] = rank_counts.get(card.rank, 0) + 1

        # Identify and remove sets of 4 cards
        for rank, count in rank_counts.items():
            if count == 4:
                sets += 1
                # self._hand = deque([card for card in self._hand if card.rank != rank])
                self._sets.add(rank)

    @property
    def hand(self):
        return self._hand

    @property
    def sets(self):
        return self._sets

    def add_card(self, card: Card):
        self._hand.append(card)

    def remove_cards(self, rank: CardRank):
        removed_cards = []
        for card in list(self._hand):
            if card.rank == rank:
                removed_cards.append(card)
                self._hand.remove(card)

        return removed_cards

    def get_random_card(self):
        if not self._hand:
            raise ValueError("Cannot get a card from an empty hand")
        return random.choice([card for card in self._hand if card.rank not in self._sets])

    def __bool__(self):
        return bool(self._hand)

    def __str__(self):
        return f"Hand: {[str(x) for x in self._hand]}\nSets: {self._sets}"

In [24]:
# Example: Explicit str conversion when printing a Player object
player = Player(
    hand=deque([Card(CardRank.TWO, CardSuit.HEARTS), Card(CardRank.THREE, CardSuit.DIAMONDS)])
)
print(str(player))  # Ensures __str__ is used


Hand: ['2♥', '3♦']
Sets: set()


In [25]:
class Simulation:
    def __init__(self, num_players: int = 2, cards_per_player: int = 7):
        self.num_players = num_players
        self.cards_per_player = cards_per_player
        self.winner = None

    # Create a full deck
    def create_deck(self) -> Deque[Card]:
        return deque([Card(rank, suit) for rank in CardRank for suit in CardSuit])

    # Shuffle the deck
    def shuffle_deck(self, deck) -> Deque[Card]:
        temp_list = list(deck)
        random.shuffle(temp_list)
        return deque(temp_list)

    # Deal cards to players
    def deal_cards(self, deck: Deque[Card]) -> Deque[Card]:
        hands = [deque() for _ in range(self.num_players)]
        for _ in range(self.cards_per_player):
            for hand in hands:
                if deck:  # Check if the deck is not empty
                    hand.append(deck.popleft())  # Deal one card from the deck

        return hands

    def random_excluding(self, number: int, exclude: int) -> int:
        # Create a list excluding the `exclude` number
        choices = [i for i in range(number) if i != exclude]
        # Randomly select a number from the filtered list
        return random.choice(choices)

    def run(self):
        deck: Deque[Card] = self.create_deck()
        deck = self.shuffle_deck(deck)

        players = [Player(hand=hand) for hand in self.deal_cards(deck=deck)]

        for index, player in enumerate(players):
            print(f"Player {index+1}")
            print(str(player))

        turn = 0
        while deck:
            current_player_idx = turn % self.num_players
            current_player = players[current_player_idx]
            current_player.count_sets()

            print(f"\nCards left in Deck: {len(deck)}")

            print(f"\nPlayer {current_player_idx + 1}'s turn.")
            print(str(current_player))

            if len(current_player.sets) * 4 == len(current_player.hand):
                current_player.add_card(deck.popleft())

            target_player_index = self.random_excluding(self.num_players, current_player_idx)
            target_player = players[target_player_index]

            requested_rank = current_player.get_random_card().rank
            print(f"Asking for {requested_rank.value} from Player {target_player_index + 1}")

            removed_cards = target_player.remove_cards(requested_rank)

            if not removed_cards:
                print("Go Fish!")
                current_player.add_card(deck.popleft())
                turn += 1

            for card in removed_cards:
                current_player.add_card(card)

            print(f"Got cards: {[str(card) for card in removed_cards]}")
        else:
            for player in players:
                player.count_sets()

        # Determine winner
        print("\nGame over!")
        player_sets = []
        for i, player in enumerate(players):
            player_sets.append(len(player.sets))
            print(f"Player {i + 1} collected {len(player.sets)} set(s).")
            print([rank.value for rank in player.sets])

        max_sets = max(player_sets)
        winners = [i + 1 for i, sets in enumerate(player_sets) if sets == max_sets]

        if len(winners) > 1:
            print(f"It's a tie between players {', '.join(map(str, winners))}!")
        else:
            print(f"Player {winners[0]} wins!")


In [36]:
sim = Simulation(cards_per_player=20)
sim.run()

Player 1
Hand: ['10♠', '8♠', 'J♦', '3♣', '2♣', 'A♦', '2♥', '4♠', '8♣', '4♦', 'Q♣', 'A♠', '7♠', '10♥', '3♥', '4♥', '2♦', '9♠', '8♥', 'K♠']
Sets: set()
Player 2
Hand: ['K♦', '5♠', '3♦', '7♦', 'K♥', '6♠', '10♦', 'A♣', '6♦', 'Q♠', 'J♣', '9♣', 'J♠', '9♦', '9♥', '2♠', '6♣', '4♣', '5♣', '10♣']
Sets: set()

Cards left in Deck: 12

Player 1's turn.
Hand: ['10♠', '8♠', 'J♦', '3♣', '2♣', 'A♦', '2♥', '4♠', '8♣', '4♦', 'Q♣', 'A♠', '7♠', '10♥', '3♥', '4♥', '2♦', '9♠', '8♥', 'K♠']
Sets: set()
Asking for 2 from Player 2
Got cards: ['2♠']

Cards left in Deck: 12

Player 1's turn.
Hand: ['10♠', '8♠', 'J♦', '3♣', '2♣', 'A♦', '2♥', '4♠', '8♣', '4♦', 'Q♣', 'A♠', '7♠', '10♥', '3♥', '4♥', '2♦', '9♠', '8♥', 'K♠', '2♠']
Sets: {<CardRank.TWO: '2'>}
Asking for K from Player 2
Got cards: ['K♦', 'K♥']

Cards left in Deck: 12

Player 1's turn.
Hand: ['10♠', '8♠', 'J♦', '3♣', '2♣', 'A♦', '2♥', '4♠', '8♣', '4♦', 'Q♣', 'A♠', '7♠', '10♥', '3♥', '4♥', '2♦', '9♠', '8♥', 'K♠', '2♠', 'K♦', 'K♥']
Sets: {<CardRank.TWO: '2'>}

In [27]:
# Create a full deck
def create_deck():
    return deque([Card(rank, suit) for rank in CardRank for suit in CardSuit])


# Shuffle the deck
def shuffle_deck(deck):
    temp_list = list(deck)
    random.shuffle(temp_list)
    return deque(temp_list)


# Deal cards to players
def deal_cards(deck, player, cards_per_player):
    for _ in range(cards_per_player):
        if deck:  # Check if the deck is not empty
            player.add_cards([deck.popleft()])  # Deal one card from the deck

In [28]:
# Play game


def play_go_fish(num_players: int, cards_per_player: int):
    deck = create_deck()
    deck = shuffle_deck(deck)

    players = []
    for _ in range(num_players):
        player = Player()
        deal_cards(deck, player, cards_per_player)
        players.append(player)

    # debug
    for index, player in enumerate(players):
        print(f"Player {index+1}")
        print(str(player))

    # Game loop
    turn = 0

    while deck or any(players):
        current_player_idx = turn % num_players
        current_player = players[current_player_idx]

        current_player_ranks = [card.rank.value for card in current_player.hand]

        current_player.count_sets()
        print(f"\nPlayer {current_player_idx + 1}'s turn.")
        print(str(current_player))

        # Check if current player has cards
        if not current_player:
            deal_cards(deck, current_player, cards_per_player)

        # requested_rank = 0
        # while not requested_rank:
        #     # Ask for a rank
        #     if current_player:
        #         requested_rank = input("Ask for a rank (e.g., 2, J, A): ").strip().upper()
        #         if requested_rank not in current_player_ranks:
        #             print(f"Invalid rank asked. You must have atleast one of {requested_rank}.")
        #             requested_rank = 0
        requested_rank = current_player.get_random_card().rank.value

        # target_player_idx = num_players
        # while target_player_idx == current_player_idx or target_player_idx >= num_players:
        #     target_player_idx = (
        #         int(input(f"Choose a player to ask (1-{num_players}, not yourself): ")) - 1
        #     )
        target_player_idx = random_excluding(num_players, current_player_idx)

        target_player = players[target_player_idx]
        print()
        print(f"Asking Target Player: {target_player_idx+1} for {requested_rank}")
        print(str(target_player))
        print()

        # Check if target player has the requested rank
        matching_cards = target_player.remove_cards(requested_rank)
        print("Handed over requested card(s).")
        print(f"Target Player: {target_player_idx+1}")
        print(str(target_player))

        if not matching_cards:
            print("Go Fish!")
            turn += 1
            if deck:
                current_player.add_cards([deck.popleft()])
        else:
            current_player.add_cards(matching_cards)
            current_player.count_sets()

        print()
        print(f"Current Player: {current_player_idx+1}")
        print(str(current_player))

    # Determine winner
    print("\nGame over!")
    player_sets = []
    for i, player in enumerate(players):
        player_sets.append(len(player.sets))
        print(f"Player {i + 1} collected {len(player.sets)} set(s).")
        print(player.sets)

    max_sets = max(player_sets)
    winners = [i + 1 for i, sets in enumerate(player_sets) if sets == max_sets]

    if len(winners) > 1:
        print(f"It's a tie between players {', '.join(map(str, winners))}!")
    else:
        print(f"Player {winners[0]} wins!")


In [29]:
players = play_go_fish(2, 7)

AttributeError: 'Player' object has no attribute 'add_cards'